Now that we have build our model, let's make future predictions. As a reminder, our plan of action was as follows:

1. Perform EDA on the dataset to extract valuable insight about the process generating the time series (COMPLETED).
2. Build a baseline model (univariable model without exogenous variables) for benchmarking purposes (COMPLETED).
3. Build a univariate model with all exogenous variables to check best possible performance (COMPLETED).
4. Evaluate the model with exogenous variables and discuss any potential issues (COMPLETED).
5. Overcome issues identified above (COMPLETED).
6. Make future predictions with the best model.
7. Replicate flow with Automated Time Series Modeling (AutoML)

In [ ]:
def what_is_installed():
    import pycaret
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except:
    !pip install pycaret-ts-alpha
    what_is_installed()

In [ ]:
import numpy as np
import pandas as pd
from pycaret.datasets import get_data
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

In [ ]:
data = get_data("airquality", verbose=False)
data["index"] = pd.to_datetime(data["Date"] + " " + data["Time"])
data.drop(columns=["Date", "Time"], inplace=True)
data.replace(-200, np.nan, inplace=True)
data.fillna(method="ffill", inplace=True)
data.set_index("index", inplace=True)

exog_vars = ['NOx(GT)', 'PT08.S3(NOx)', 'RH']
data = data[exog_vars]
data.head()

## Step 6: Making Future Predictions

### Step 6A: Get future exogenous variable values using forecasting

In [ ]:
exog_exps = []
exog_models = []
for exog_var in exog_vars:
    exog_exp = TimeSeriesExperiment()
    exog_exp.setup(data=data[exog_var], fh=48, session_id=42)

    # Users can customize how to model future exogenous variables i.e. add
    # more steps and models to potentially get better models at the expense
    # of higher modeling time.
    best = exog_exp.compare_models(
        sort="mase", include=["arima", "ets", "exp_smooth", "theta", "lightgbm_cds_dt",]        
    )
    final_exog_model = exog_exp.finalize_model(best)

    exog_exps.append(exog_exp)
    exog_models.append(final_exog_model)

# Step 2: Get future predictions for exog variables ----
future_exog = [
    exog_exp.predict_model(exog_model)
    for exog_exp, exog_model in zip(exog_exps, exog_models)
]
future_exog = pd.concat(future_exog, axis=1)

In [ ]:
future_exog

### Step 6B: Load Model and make future predcitons for the target variable

In [ ]:
exp_future = TimeSeriesExperiment()

In [ ]:
final_slim_model = exp_future.load_model("final_slim_model")

In [ ]:
future_preds = exp_future.predict_model(final_slim_model, X=future_exog)
future_preds.plot()